In [1]:
!pip install numpy opencv-python pillow tqdm requests moviepy

In [16]:
!pip install tweepy

In [17]:
import tweepy
import os, json, random, re, time
from urllib.parse import urlparse, parse_qs

# -----------------------
# Twitter API Credentials
# -----------------------
CLIENT_ID = 'MlBBaFNaN2NITHRISW5naEhBUTU6MTpjaQ'
CLIENT_SECRET = 'KNWRGIW5SGpMSJ2mQ2-yeAylc3IB-vj-4Ba8XDf3Cq-c1uuLfS'
# This redirect URI must be registered in your Twitter Developer App settings.
REDIRECT_URI = 'https://nexocodai.github.io/'
# Path to store OAuth token (in Colab, this is temporary)
TOKEN_FILE = 'twitter_token.json'

# -----------------------
# OAuth2 Initialization
# -----------------------
def init_twitter_client():
    if os.path.exists(TOKEN_FILE):
        with open(TOKEN_FILE, 'r') as f:
            token_data = json.load(f)
        print("Loaded access token from file.")
        # Create client using the stored access token
        client = tweepy.Client(
            access_token=token_data.get("access_token"),
            consumer_key=CLIENT_ID,
            consumer_secret=CLIENT_SECRET
        )
        return client
    else:
        # Create an OAuth2UserHandler to initiate the PKCE flow.
        oauth2_user_handler = tweepy.OAuth2UserHandler(
            client_id=CLIENT_ID,
            client_secret=CLIENT_SECRET,
            redirect_uri=REDIRECT_URI,
            scope=["tweet.read", "tweet.write", "users.read", "offline.access"]
        )
        auth_url = oauth2_user_handler.get_authorization_url()
        print("Open the following URL in your browser and authorize the app:")
        print(auth_url)
        callback_url = input("Paste the full callback URL after authorization: ").strip()

        # Parse the callback URL for the code and state parameters.
        parsed = urlparse(callback_url)
        query_params = parse_qs(parsed.query)
        code = query_params.get('code', [None])[0]
        returned_state = query_params.get('state', [None])[0]

        if not code or not returned_state:
            raise Exception("Missing code or state parameter in callback URL.")

        token = oauth2_user_handler.fetch_token(callback_url)
        with open(TOKEN_FILE, 'w') as f:
            json.dump(token, f, indent=2)
        print("Access token obtained and saved.")
        client = tweepy.Client(
            access_token=token.get("access_token"),
            consumer_key=CLIENT_ID,
            consumer_secret=CLIENT_SECRET
        )
        return client

client = init_twitter_client()

# -----------------------
# Book Storage Setup
# -----------------------
BOOK_STORAGE_DIR = "book_storage"
os.makedirs(BOOK_STORAGE_DIR, exist_ok=True)

def create_sample_book_file():
    sample_book_path = os.path.join(BOOK_STORAGE_DIR, "book_1.json")
    if not os.path.exists(sample_book_path):
        sample_book = {
            "title": "Quantum Corsairs: Nebula's Edge",
            "author": "Jax Meridian",
            "synopsis": (
                "Dive into the neon-drenched future where rogue starship captains, the Quantum Corsairs, defy galactic law. "
                "Captain Ryla, haunted by a past betrayal, navigates treacherous nebula storms and corporate assassins to uncover a "
                "hidden AI artifact promising unimaginable power. But wielding it means confronting her deepest fears and sparking "
                "a rebellion against the iron-fisted Galactic Federation. Prepare for pulse-pounding space battles, cyberpunk intrigue, "
                "and a fight for freedom at the galaxy's edge, where destiny hangs in the balance and every choice is a gamble."
            ),
            "font": {
                "name": "Roboto Mono",
                "url": "https://fonts.google.com/specimen/Roboto+Mono"
            }
        }
        with open(sample_book_path, 'w') as f:
            json.dump(sample_book, f, indent=2)
        print("Created sample book file.")

def read_book_data():
    books = []
    for filename in os.listdir(BOOK_STORAGE_DIR):
        if filename.endswith('.json'):
            path_file = os.path.join(BOOK_STORAGE_DIR, filename)
            with open(path_file, 'r') as f:
                book_data = json.load(f)
            book_data["id"] = filename.replace(".json", "")
            books.append(book_data)
    return books

# -----------------------
# Tweet Generation Helpers
# -----------------------
def extract_key_elements(synopsis):
    sentences = re.split(r'[.!?]+', synopsis)
    sentences = [s.strip() for s in sentences if s.strip()]
    character_regex = r'\b[A-Z][a-z]+\b'
    characters = []
    for sentence in sentences:
        matches = re.findall(character_regex, sentence)
        if matches:
            characters.extend(matches)
    characters = list(set([c for c in characters if len(c) > 3]))
    theme_words = ['galaxy', 'space', 'nebula', 'magic', 'power', 'artifact', 'war', 'rebellion',
                   'empire', 'federation', 'captain', 'adventure', 'battle', 'mission', 'quest',
                   'journey', 'secret', 'ancient', 'future', 'destiny', 'alien', 'robot', 'AI']
    found_themes = [theme for theme in theme_words if theme.lower() in synopsis.lower()]
    hook = sentences[0] if sentences else ""
    if len(hook) > 60:
        hook = hook[:57] + "..."
    dramatic_moment = ""
    for sentence in sentences:
        if (("?" in sentence) or ("must" in sentence.lower()) or
            ("danger" in sentence.lower()) or ("fate" in sentence.lower()) or ("risk" in sentence.lower())) \
            and len(sentence) < 100:
            dramatic_moment = sentence
            break
    return {
        "characters": characters[:3],
        "themes": found_themes[:5],
        "hook": hook,
        "dramatic_moment": dramatic_moment
    }

def generate_hashtags(book, elements):
    genre_hashtags = ['#SciFi', '#ScienceFiction', '#SpaceOpera', '#SpaceAdventure',
                      '#SpeculativeFiction', '#AIFiction', '#CyberpunkFiction',
                      '#FuturisticFiction', '#GalacticAdventure', '#SpaceSaga']
    content_hashtags = ['#AICreatedFiction', '#GenerativeStorytelling', '#DigitalFiction',
                        '#AIStories', '#AIWorldbuilding', '#GeneratedUniverse',
                        '#CreativeTech', '#FutureFiction', '#TechLit', '#ImaginaryWorlds']
    num_hashtags = random.randint(2, 3)
    genre_count = min(random.randint(1, num_hashtags), len(genre_hashtags))
    content_count = num_hashtags - genre_count
    selected_genre = random.sample(genre_hashtags, genre_count)
    selected_content = random.sample(content_hashtags, content_count) if content_count > 0 else []
    return " ".join(selected_genre + selected_content)

def generate_tweet(book):
    elements = extract_key_elements(book["synopsis"])
    hashtags = generate_hashtags(book, elements)
    emojis = ['✨', '🚀', '🌌', '🔮', '⚡', '🌠', '💫', '🪐', '🌍', '🔭', '📚']
    def random_emoji():
        return random.choice(emojis)
    templates = [
        f'{random_emoji()} "{book["title"]}" {random_emoji()}\n\n{elements["hook"]}\n\nA new AI-generated universe awaits! {hashtags}',
        f'From the digital pages of "{book["title"]}":\n\n"{elements["dramatic_moment"]}"\n\n{hashtags}',
        (f'Meet {elements["characters"][0]}, navigating a universe of {" and ".join(elements["themes"][:2])} in the AI-created story "{book["title"]}" '
         f'{random_emoji()} {hashtags}') if elements["characters"] else f'Explore a universe of {" and ".join(elements["themes"][:2])} in the AI-created story "{book["title"]}" {random_emoji()} {hashtags}',
        f'{random_emoji()} In "{book["title"]}", {", ".join(elements["themes"][:3])} collide to create an unforgettable digital universe. {hashtags}',
        f'"{book["synopsis"].split(".")[0].strip()}." \n\nDiscover more in "{book["title"]}", an AI-crafted adventure. {hashtags}',
        f'What would you do if you discovered {elements["themes"][0]} that could change everything? Step into "{book["title"]}" and find out! {random_emoji()} {hashtags}',
        f'{random_emoji()} AI-generated universe spotlight: "{book["title"]}"\n\nWhere {" meets ".join(elements["themes"][:2])} in a digital frontier of possibilities. {hashtags}',
        f'{random_emoji()} NEW AI STORY UNIVERSE {random_emoji()}\n\n"{book["title"]}"\n\n{book["synopsis"][:100]}... {hashtags}',
        f'Explore the {elements["themes"][0]} and {elements["themes"][1] if len(elements["themes"]) > 1 else "adventure"} in an AI-crafted universe. "{book["title"]}" - where imagination meets algorithm. {hashtags}',
        f'{random_emoji()} Just released: "{book["title"]}" by AI author collaborative.\n\nDive into a {elements["themes"][0] if elements["themes"] else "space"} adventure unlike any other! {hashtags}'
    ]
    tweet = random.choice(templates)
    if len(tweet) > 280:
        tweet = tweet[:277] + "..."
    return tweet

def post_tweet(tweet_text, media_path=None):
    try:
        # For simplicity, this example does not handle media uploads.
        response = client.create_tweet(text=tweet_text)
        tweet_id = response.data.get("id")
        print("Tweet posted successfully. Tweet ID:", tweet_id)
        return True
    except Exception as e:
        print("Error posting tweet:", e)
        return False

def generate_and_post_tweets(count=1):
    books = read_book_data()
    if not books:
        print("No book data found. Please add JSON files to the book storage directory.")
        return
    for i in range(count):
        book = random.choice(books)
        tweet_text = generate_tweet(book)
        print(f"\nPosting tweet {i+1}/{count}:\n{tweet_text}")
        post_tweet(tweet_text)
        if i < count - 1:
            time.sleep(2)
    print(f"\nFinished posting {count} tweet(s).")

# -----------------------
# Simple CLI for Interaction
# -----------------------
def cli():
    while True:
        print("\nAI Fiction Tweet Publisher CLI")
        print("-----------------------------")
        print("1. Post tweets now")
        print("2. Post a custom tweet")
        print("3. View available books")
        print("4. Generate sample tweet previews")
        print("5. Exit")
        option = input("Select an option: ").strip()
        if option == "1":
            count = input("How many tweets to post? (default: 1): ").strip()
            try:
                count = int(count)
            except:
                count = 1
            generate_and_post_tweets(count)
        elif option == "2":
            tweet_text = input("Enter tweet text: ").strip()
            media = input("Add media? Enter file path or leave empty: ").strip()
            post_tweet(tweet_text, media if media else None)
        elif option == "3":
            books = read_book_data()
            print("\nAvailable Books:")
            if not books:
                print("No books found. Add JSON files to the storage directory.")
            else:
                for i, book in enumerate(books, 1):
                    print(f"\n{i}. {book['title']} by {book['author']}")
                    print(f"   ID: {book.get('id', 'N/A')}")
                    print(f"   Synopsis: {book['synopsis'][:100]}...")
        elif option == "4":
            books = read_book_data()
            if not books:
                print("No books found for preview.")
            else:
                for book in books:
                    print(f"\n--- Tweets for: {book['title']} ---")
                    for _ in range(3):
                        print("\n" + generate_tweet(book))
        elif option == "5":
            print("Exiting...")
            break
        else:
            print("Invalid option. Please try again.")

# -----------------------
# Run Setup & CLI
# -----------------------
create_sample_book_file()
cli()


Open the following URL in your browser and authorize the app:
https://twitter.com/i/oauth2/authorize?response_type=code&client_id=MlBBaFNaN2NITHRISW5naEhBUTU6MTpjaQ&redirect_uri=https%3A%2F%2Fnexocodai.github.io%2F&scope=tweet.read+tweet.write+users.read+offline.access&state=CoSwctl9z90FnKwngt9DszJM3BdYoH&code_challenge=M2lqLJdNQhAckhgMeh16u9OYcU4s2EZC2yALBj6oRVw&code_challenge_method=S256
Paste the full callback URL after authorization: https://nexocodai.github.io/?state=CoSwctl9z90FnKwngt9DszJM3BdYoH&code=d3NwOWVmZ0xJcDVWOW5lbFpXclFUa19iVjMwUU1QdWs4d2dtcHNtRTR6TVY4OjE3NDEyMjU3ODUzNjg6MToxOmFjOjE
Access token obtained and saved.
Created sample book file.

AI Fiction Tweet Publisher CLI
-----------------------------
1. Post tweets now
2. Post a custom tweet
3. View available books
4. Generate sample tweet previews
5. Exit
Select an option: 1
How many tweets to post? (default: 1): 1

Posting tweet 1/1:
Meet Quantum, navigating a universe of galaxy and space in the AI-created story "Quan

KeyboardInterrupt: Interrupted by user